The goal of this assignment was to use Hadoop Streaming to calculate  
the percentage of words in a master text file (en_articles_part) that come  
from a smaller text file (stop_words_en.txt)

In [ ]:
%%writefile mapper.py

import sys
import re

reload(sys)
sys.setdefaultencoding('utf-8')

with open('stop_words_en.txt') as sf:
    content = sf.readlines()
    stopwords = set(wd.strip().lower() for wd in content)

for line in sys.stdin:
    try:
        article_id, text = unicode(line.strip()).split('\t', 1)
    except ValueError as e:
        continue
    words = re.split("\W*\s+\W*", text, flags=re.UNICODE)
    for word in words:
        print >> sys.stderr, "reporter:counter:Wiki stats,Total words,%d" % 1
        print "TOTALWORDS\t%d" % 1
        
        if word in stopwords:
            print >> sys.stderr, "reporter:counter:Wiki stats,Stop words,%d" % 1
            print "STOPWORDS\t%d" % 1           

In [ ]:
%%writefile reducer.py

import sys

result = {'TOTALWORDS': 0, 'STOPWORDS':0}

for line in sys.stdin:
    try:
        key, count = line.strip().split('\t', 1)
        count = int(count)
    except ValueError as e:
        continue
    
    if key:
        result[key] = result[key] + count

print(float(result['STOPWORDS']) / result['TOTALWORDS'] * 100)

In [ ]:
%%bash

OUT_DIR="stop_words_result"
NUM_REDUCERS=1

hdfs dfs -rm -r -skipTrash ${OUT_DIR} > /dev/null

yarn jar /opt/cloudera/parcels/CDH/lib/hadoop-mapreduce/hadoop-streaming.jar \
    -D mapred.jab.name="Streaming stop words" \
    -D mapreduce.job.reduces=${NUM_REDUCERS} \
    -files mapper.py,reducer.py,/datasets/stop_words_en.txt \
    -mapper "python mapper.py" \
    -reducer "python reducer.py" \
    -input /data/wiki/en_articles_part \
    -output ${OUT_DIR} > /dev/null

hdfs dfs -cat ${OUT_DIR}/part-00000 | head         